In [1]:
import json
import re

import pandas as pd
import numpy as np

from methods_parse.methods_parse import UsedMethods, shift_methods

In [2]:
# comp_path = '../../data_mini/competitions_info_cleaned.csv'
# competitions = pd.read_csv(comp_path)
# competitions.shape

In [3]:
graph_path = '../../data/actual_graph_2021-04-18.csv'
graph = pd.read_csv(graph_path)
graph.rename({'id':'graph_vertex_id'}, axis=1, inplace=True)
# graph.head()

In [4]:
markup_path = '../../data/markup_data_2021-04-18.csv'
markup_data = pd.read_csv(markup_path)
markup_data.shape

(2610, 8)

In [5]:
markup_data = markup_data.merge(graph, on='graph_vertex_id', how='left')

In [6]:
# !python comment_cleanup

In [7]:
# !python script_shift_up_down.py: graph_vertex_m1, graph_vertex_m2, graph_vertex_m3, graph_vertex_p1, graph_vertex_p2, graph_vertex_p3, comments, libraries

In [8]:
def label_code_by_regex(code_block, regex_graph_version:int=6):
    regex_graph_dir = '../../graph/graph_v{}.txt'.format(regex_graph_version)
    with open(regex_graph_dir, "r") as regex_graph_file:
        regex_graph = json.load(regex_graph_file)
    found_vertices = []
    for i, vertex in enumerate(regex_graph):
        tokens = regex_graph[vertex]
        for token in tokens:
            result = re.search(token.replace('(','\('), code_block)
            if result!=None:
                found_vertices.append(vertex)
                break
    return found_vertices

In [13]:
def get_comments(text: str):
    text = str(text)
#     comments = comment_parser.extract_comments_from_str(text, mime="text/x-python")
#     return "\n".join([line.text() for line in comments])

    comments = []
    line_array = [line.strip() for line in text.split("\n")]
    # now get libraries if line start with "#"
    for line_ind in range(len(line_array)):
        if line_array[line_ind].startswith("#"):
            comments.append(line_array[line_ind][1:])
        elif line_array[line_ind].startswith("'''"):
            multi_comm = str()
            multi_comm += line_array[line_ind][3:] + "\n"
            if "'''" in multi_comm:
                multi_comm = multi_comm.replace("'''", "")
                line_ind += 1
                comments.append(multi_comm)
                continue
            line_ind += 1
            while line_ind < len(line_array):
                multi_comm += line_array[line_ind] + "\n"
                if "'''" in multi_comm:
                    multi_comm = multi_comm.replace("'''", "")
                    line_ind += 1
                    break
                line_ind += 1
            comments.append(multi_comm)
        elif line_array[line_ind].startswith('"""'):
            multi_comm = str()
            multi_comm += line_array[line_ind][3:] + "\n"
            if '"""' in multi_comm:
                multi_comm = multi_comm.replace('"""', "")
                line_ind += 1
                comments.append(multi_comm)
                continue
            line_ind += 1
            while line_ind < len(line_array):
                multi_comm += line_array[line_ind] + "\n"
                if '"""' in multi_comm:
                    multi_comm = multi_comm.replace('"""', "")
                    line_ind += 1
                    break
                line_ind += 1
            comments.append(multi_comm)
            pass
    return "\n".join([line for line in comments])

In [18]:
def get_libraries(text: str, dicts: tuple):
    name_lib_import = dicts[0]
    name_lib_from = dicts[1]
    text = str(text)
    libs = []
    line_array = [line.strip() for line in text.split("\n")]
    # delete all double spaces in all lines
    for ind in range(len(line_array)):
        
        prev_len = -1
        while prev_len != len(line_array[ind]):
            prev_len = len(line_array[ind])
            line_array[ind] = line_array[ind].replace("  ", " ")
    # now get libraries if line start with "import" or "from"
    for line in line_array:
        if line.startswith("import") or line.startswith("from"):
            libs.append(line.split(" ")[1].split(".")[0])
    for name in name_lib_from.keys():
        for line in line_array:
            if name in line:
                libs.append(name_lib_from[name])
                
    for name in name_lib_import.keys():
        for line in line_array:
            if name + "." in line:
                libs.append(name_lib_import[name])
        
    return "\n".join(list(set(libs)))

In [19]:
def process_row(row):
    '''
    methods_parse.py: python_methods, python_methods_m1, python_methods_m2, python_methods_m3, python_methods_p1, python_methods_p2, python_methods_p3
    models_scripts/regex.py: graph_vertex_regex
    '''
    row['python_methods'] = UsedMethods(row['code_block'])
    row['graph_vertex_regex'] = label_code_by_regex(row['code_block'])
    row['comments'] = get_comments(row['code_block'])
    # row['libraries'] = get_libraries(row['code_block'], get_dict(temp_df.code_block.values))
    return row

In [20]:
markup_data = markup_data.apply(process_row, axis=1)

TypeError: get_libraries() missing 1 required positional argument: 'dicts'

In [ ]:
markup_data = shift_methods(markup_data, shift_range=5)

In [ ]:
markup_data.head(25)